[Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification)

# 导入工具

In [1]:
import pandas as pd
import numpy as np
import sys

In [8]:
sys.setdefaultencoding('utf-8')

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

In [53]:
pd.set_option('display.max_colwidth', 20)

In [54]:
df = pd.DataFrame(index=np.array(range(1)),columns=['Object', 'Verb', 'Comment'])
df.iloc[:,] = '我', '说','你特么再说一遍？你特么再说一遍？你特么再说一遍？你特么再说一遍？'

In [55]:
df

,Object,Verb,Comment
0,我,说,你特么再说一遍？你特么再说一遍？...


ValueError: Need to invoke asoption_context(pat, val, [(pat, val), ...)).